In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from EVCS_GA_FCFS import EVCSIndividual
import copy
#from DW import Draw
import random
import time


class EVCSIndividual:

    '''
    indiviual of demand oriented method
    '''
    def __init__(self,data,ev_ct,ec_cp,ev_n):

        self.data=data  #候选数据
        self.evcs=np.zeros(ev_n)  #每个充电站的车数
        self.indiviual=[]
        self.fitness=[]
        self.ev_ct=ev_ct
        self.ev_cp=ec_cp
        self.ev_n=ev_n

    def GenerateIndividual(self):
        '''
        generate a random chromsome for genetic algorithm
        '''
        gene = list()    #问题的解，基因，种群中的个体：[0，...，city_size]
        for i in range(len(self.data)):
            ind=random.choices(self.data[i])
            copy_ind=copy.deepcopy(ind[0])
            copy_ind.append(i)
            gene.append(copy_ind)
        self.indiviual=gene

    def calculateFitness(self,cd_n=5):
        '''
        calculate the fitness of the indiviual
        '''
        #distance = 0.0
        ev_n=np.zeros((self.ev_n,cd_n))  #单位时间内每个电桩充电车数
        cd_wt=np.zeros((self.ev_n,cd_n)) #充电桩工作总时间
        cd_st=np.zeros((self.ev_n,cd_n)) #充电桩空闲时间
        st=0.   #电桩空闲总时间  
        qt=0.   #电车排队总时间 
        tt=0.   #充电总时间 到充电桩时间+排队时间+充电时间     
        cost=0. #充电总成本成本
        n=[]  
        x_n=0
        for i in range(len(self.indiviual)): #len(self.indiviual)
            cost+=self.indiviual[i][1]+self.ev_cp[self.indiviual[i][0]]*self.ev_ct[self.indiviual[i][2]]
            k=int(self.evcs[self.indiviual[i][0]]%cd_n)
            #print(k)
            self.evcs[self.indiviual[i][0]]+=1
            if cd_wt[self.indiviual[i][0]][k]<60:
                ev_n[self.indiviual[i][0]][k]+=1
            if self.indiviual[i][1]<cd_wt[self.indiviual[i][0]][k]:
                tt+=cd_wt[self.indiviual[i][0]][k]+self.ev_ct[self.indiviual[i][0]]
                qt+=cd_wt[self.indiviual[i][0]][k]-self.indiviual[i][1]
                #print("第%d辆排队的EV，车辆编号：%d,具体信息：%s,桩号：%d,需要排队时间：%f" %(x_n,i,str(self.indiviual[i]),k,cd_wt[self.indiviual[i][0]][k]-self.indiviual[i][1]))
                cd_wt[self.indiviual[i][0]][k]=cd_wt[self.indiviual[i][0]][k]+self.ev_ct[self.indiviual[i][0]]
                x_n+=1
            else:
                tt+=self.indiviual[i][1]+self.ev_ct[self.indiviual[i][0]]
                st+=self.indiviual[i][1]-cd_wt[self.indiviual[i][0]][k]
                cd_st[self.indiviual[i][0]][k]+=self.indiviual[i][1]-cd_wt[self.indiviual[i][0]][k]
                cd_wt[self.indiviual[i][0]][k]=self.indiviual[i][1]+self.ev_ct[self.indiviual[i][0]]
        revnue=0.
        t_ev_n=0
        t_st_v=0
        for i in range(self.ev_n):
            for j in range(cd_n):
                revnue+=(cd_wt[i][j]-cd_st[i][j])*self.ev_cp[i]
                t_ev_n+=ev_n[i][j]
                t_st_v+=cd_st[i][j]
        
        n.append(revnue)      #总收益  
        n.append(cost)        #总成本
        n.append(tt)          #总时间
        n.append(qt)          #总排队时间
        n.append(st)          #总空闲时间
        n.append(t_ev_n)      #单位时间每个充电桩充电电车总数
       # n.append(t_st_v)
        self.fitness=np.array(n)
        
        
class GeneticAlgorithm:
    #print("ga")
    def __init__(self, t_c_l, ev_ct, ev_cp, n,c_rate=0.7, m_rate=0.3, pop_size=200, maxnum=200):
        self.pop_size = pop_size#50
        self.fitness = np.zeros(self.pop_size)#Pop_size 为50，种群个数，应该修改为参数，可传递任意种群个数
        self.c_rate = c_rate #交叉率，0.7，修改为可变参数
        self.m_rate = m_rate #变异率，0.05，修改为
        self.maxiternum = maxnum# 最大迭代次数
        self.population=[]
        self.bestfitness=0.   #最好的适应值
        self.besttruefitness=[] # 最好的真适应值
        self.bestIndex=0
        self.bestgene=np.array([])
        #self.dw = Draw()    #绘图类
        self.trace = np.zeros((self.maxiternum, 2))
        self.avefitness=0.
        self.data=t_c_l
        self.ev_ct=ev_ct
        self.ev_cp=ev_cp
        self.cs_n=n
        self.individual=[]   #最好的个体
    
    def initialize(self):
        '''
        initialize the population of GA
        '''
        for i in range(0, int(self.pop_size)):
            ind = EVCSIndividual(self.data,self.ev_ct,self.ev_cp,self.cs_n)
            ind.GenerateIndividual()
            self.population.append(ind)
    
        
    def evaluation(self):
        '''
        evaluation the fitness of the population
        '''
        for i in range(0, int(self.pop_size)):
            self.population[i].calculateFitness()
            #self.fitness[i] = self.population[i].fitness
            self.fitness[i] = 0.3*(self.population[i].fitness[3]/40000)+0.4*(self.population[i].fitness[4]/3000)+0.3*(1-self.population[i].fitness[5]/899)
            
    def selection(self):
        for i in range(self.pop_size):
            if i != self.bestIndex and self.fitness[i] > self.avefitness:
                #print(i,self.bestIndex)
                pi = self.cross(self.population[self.bestIndex].indiviual, self.population[i].indiviual)
                self.population[i].indiviual= self.mutate(pi)
                #print(self.population[i].indiviual)
                self.population[i].calculateFitness()
                #self.fitness[i] = self.population[i].fitness
                self.fitness[i] = 0.3*(self.population[i].fitness[3]/40000)+0.4*(self.population[i].fitness[4]/3000)+0.3*(1-self.population[i].fitness[5]/899)
    
    def crossoverMutation(self):
         for j in range(self.pop_size):
                #if j != self.bestIndex:
                r = np.random.randint(0, self.pop_size - 1)
                if j != r and j != self.bestIndex:
                    #print(j,self.bestIndex)
                    nind = self.cross(self.population[j].indiviual, self.population[r].indiviual)    #交叉种群中第j,r个体的基因
                    self.population[j].indiviual = self.mutate(nind)    #突变种群中第j个体的基因
                    self.population[j].calculateFitness()
                    #self.fitness[j] = self.population[j].fitness
                    self.fitness[j] = 0.3*(self.population[j].fitness[3]/40000)+0.4*(self.population[j].fitness[4]/3000)+0.3*(1-self.population[j].fitness[5]/899)
    
    def cross(self, parent1, parent2):
        """交叉p1,p2的部分基因片段"""
        if np.random.rand() > self.c_rate:
            return parent1
        index1 = np.random.randint(0, len(parent1) - 1)
        index2 = np.random.randint(index1,  len(parent1)- 1)
        parent1[index1:index2]=parent2[index1:index2]
        return parent1
    
    def mutate(self,gene):
        """突变"""
        if np.random.rand() > self.m_rate:
            return gene
        #newGene = self.cross(gene, self.population[self.bestIndex].indiviual)
        #newGene = np.array(newGene)
        index1 = np.random.randint(0, len(gene) - 1)
        index2 = np.random.randint(index1,  len(gene)- 1)
        for i in range(index1,index2+1):
            ind=random.choices(self.data[i])
            copy_ind=copy.deepcopy(ind[0])
            copy_ind.append(i)
            gene[i]=copy_ind
        return gene
     
    def solve(self):
        self.t = 0
        self.initialize()
        self.evaluation()
        self.bestfitness = np.min(self.fitness)
        self.bestIndex = np.argmin(self.fitness)
        self.bestgene = copy.deepcopy(self.population[self.bestIndex])
        self.besttruefitness=self.population[self.bestIndex].fitness
        self.individual=self.population[self.bestIndex].indiviual
        self.avefitness = np.mean(self.fitness)
        self.trace[self.t, 0] = self.fitness[self.bestIndex]
        self.trace[self.t, 1] = self.avefitness
        print("%d次迭代: 适应值:%f,平均适应值:%f,最优结果:%s" % (
            self.t+1, self.trace[self.t, 0], self.trace[self.t, 1],str(self.bestgene.fitness)))
        while self.t < self.maxiternum - 1:
            self.t += 1
            self.selection()
            self.crossoverMutation()
            localbest = np.min(self.fitness)
            self.bestIndex = np.argmin(self.fitness)
            if localbest<self.bestfitness:
                self.bestfitness=localbest
                self.besttruefitness=self.population[self.bestIndex].fitness
                self.individual=self.population[self.bestIndex].indiviual           
            self.bestgene = copy.deepcopy(self.population[self.bestIndex])
            self.avefitness = np.mean(self.fitness)            
            self.trace[self.t, 0] = self.fitness[self.bestIndex]
            self.trace[self.t, 1] = self.avefitness
            print("%d次迭代: 适应值:%f,平均适应值:%f,最优结果:%s" % (
                self.t+1, self.trace[self.t, 0], self.trace[self.t, 1],str(self.bestgene.fitness)))
        print("True Best Fitness:",self.besttruefitness)
        print("Best Fitness:",self.bestfitness)
        print("Average of queuing time (minute):",self.besttruefitness[3]/899)
        print("Average of idle time (minute):",self.besttruefitness[4]/170)
        print("Number of charged EV within an hour:",int(self.besttruefitness[5]))
        print("Total cost of all EV:",self.besttruefitness[1])
        print("Total revenue of EVCS:",self.besttruefitness[0])
        
def main():
    '''读取充电站数据'''
    data = pd.read_csv("chargingstations.csv", delimiter=";", header=None).values  #读取34省市数据，以';'分割，dataframe
    cities = data[:, 1:]# 0行至最后一行，1列至最后一列
    cities_name = data[:, 0] #0行至最后一行，第0列
    city_size = data.shape[0]# shape属性返回包含行和列的元组
    locations = np.arange(cities.shape[0])    #问题的解，基因，种群中的个体：[0，...，city_size]

    ev_data_1000= pd.read_csv('ev_data_1000.csv').values 

    ev_x=ev_data_1000[:, 1:2]
    ev_x=[j for i in ev_x for j in i]
    ev_y=ev_data_1000[:, 2:3]
    ev_y=[j for i in ev_y for j in i]
    ev_ld=ev_data_1000[:, 3:4]
    ev_ld=[j for i in ev_ld for j in i]
    ev_ct=ev_data_1000[:, 4:5]
    ev_ct=[j for i in ev_ct for j in i]
    ev_cp_34= pd.read_csv('ev_cp_34.csv').values 
    ev_cp=ev_cp_34[:,1:]
    ev_cp=[j for i in ev_cp for j in i]

    '''电车候选电站'''
    t_c_l=[] #每个电车的候选充电站
    for i in range(1000):
        c_l=[]
        for j in range(34):
            d=np.sqrt((ev_x[i] - cities[j][0]) ** 2 + (ev_y[i] - cities[j][1]) ** 2)
            if d<ev_ld[i]:
                c=[]
                c.append(int(j))      # 充电站序号
                c.append(d)      # 距离充电站距离，也是行驶时间
                c_l.append(c)
        t_c_l.append(c_l)        

    for i in range(len(t_c_l)-1,-1,-1):
        if t_c_l[i]==[]:
            del ev_x[i]
            del ev_y[i]
            del ev_ld[i]
            del ev_ct[i]
            del t_c_l[i]
            
            
    GA=GeneticAlgorithm(t_c_l,ev_ct,ev_cp,len(cities))
    GA.solve()
    
if __name__ == '__main__':
    start=time.time()
    main()
    end=time.time()
    print("Time consuming:",end-start)

1次迭代: 适应值:0.614973,平均适应值:0.660070,最优结果:[16588.71047892 28249.13091019 56020.43133002 31454.94813738
  2169.71120333   630.        ]
2次迭代: 适应值:0.613609,平均适应值:0.646186,最优结果:[16848.99756779 28385.29582842 56780.74154893 32064.9758617
  2117.65678483   627.        ]
3次迭代: 适应值:0.613609,平均适应值:0.658200,最优结果:[16848.99756779 28385.29582842 56780.74154893 32064.9758617
  2117.65678483   627.        ]
4次迭代: 适应值:0.613609,平均适应值:0.654715,最优结果:[16848.99756779 28385.29582842 56780.74154893 32064.9758617
  2117.65678483   627.        ]
5次迭代: 适应值:0.613609,平均适应值:0.660284,最优结果:[16848.99756779 28385.29582842 56780.74154893 32064.9758617
  2117.65678483   627.        ]
6次迭代: 适应值:0.613609,平均适应值:0.649439,最优结果:[16848.99756779 28385.29582842 56780.74154893 32064.9758617
  2117.65678483   627.        ]
7次迭代: 适应值:0.613609,平均适应值:0.653623,最优结果:[16848.99756779 28385.29582842 56780.74154893 32064.9758617
  2117.65678483   627.        ]
8次迭代: 适应值:0.613609,平均适应值:0.645446,最优结果:[16848.99756779 28385.29582842 56780.741548

63次迭代: 适应值:0.605074,平均适应值:0.652122,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
64次迭代: 适应值:0.605074,平均适应值:0.653458,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
65次迭代: 适应值:0.605074,平均适应值:0.669020,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
66次迭代: 适应值:0.605074,平均适应值:0.660006,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
67次迭代: 适应值:0.605074,平均适应值:0.662769,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
68次迭代: 适应值:0.605074,平均适应值:0.648250,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
69次迭代: 适应值:0.605074,平均适应值:0.649371,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
70次迭代: 适应值:0.605074,平均适应值:0.648404,最优结果:[16838.87990189 28226.3721102

125次迭代: 适应值:0.605074,平均适应值:0.651774,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
126次迭代: 适应值:0.605074,平均适应值:0.646783,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
127次迭代: 适应值:0.605074,平均适应值:0.648497,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
128次迭代: 适应值:0.605074,平均适应值:0.657133,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
129次迭代: 适应值:0.605074,平均适应值:0.665297,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
130次迭代: 适应值:0.605074,平均适应值:0.656519,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
131次迭代: 适应值:0.605074,平均适应值:0.665165,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
132次迭代: 适应值:0.605074,平均适应值:0.663110,最优结果:[16838.87990189 28226

187次迭代: 适应值:0.605074,平均适应值:0.661799,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
188次迭代: 适应值:0.605074,平均适应值:0.658551,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
189次迭代: 适应值:0.605074,平均适应值:0.665660,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
190次迭代: 适应值:0.605074,平均适应值:0.677515,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
191次迭代: 适应值:0.605074,平均适应值:0.649480,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
192次迭代: 适应值:0.605074,平均适应值:0.665057,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
193次迭代: 适应值:0.605074,平均适应值:0.655599,最优结果:[16838.87990189 28226.37211024 56186.38055091 31582.16307741
  2095.81861338   633.        ]
194次迭代: 适应值:0.605074,平均适应值:0.647972,最优结果:[16838.87990189 28226